<a href="https://colab.research.google.com/github/kthfighting/20161298project1/blob/master/%EC%B0%A8%EB%9F%89%EC%A7%80%EB%8A%A5%EA%B8%B0%EC%B4%88%EA%B3%BC%EC%A0%9C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Lhyejin/LaneNet-PyTorch.git

In [ ]:
%cd LaneNet-PyTorch//
!ls
!pip install -r requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os.path as ops
import numpy as np
import torch
import cv2
import time
import os
import matplotlib.pylab as plt
import sys
from tqdm import tqdm
import imageio
from dataset.dataset_utils import TUSIMPLE
from Lanenet.model2 import Lanenet
from utils.evaluation import gray_to_rgb_emb, process_instance_embedding
from google.colab.patches import cv2_imshow

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the Model
model_path = './TUSIMPLE/Lanenet_output/lanenet_epoch_39_batch_8.model'
LaneNet_model = Lanenet(2, 4)
LaneNet_model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
LaneNet_model.to(device)

def inference(gt_img_org):
    # BGR 순서
    org_shape = gt_img_org.shape
    gt_image = cv2.resize(gt_img_org, dsize=(512, 256), interpolation=cv2.INTER_LINEAR)
    gt_image = gt_image / 127.5 - 1.0
    gt_image = torch.tensor(gt_image, dtype=torch.float)
    gt_image = np.transpose(gt_image, (2, 0, 1))
    gt_image = gt_image.to(device)
    # lane segmentation 
    binary_final_logits, instance_embedding = LaneNet_model(gt_image.unsqueeze(0))
    binary_final_logits, instance_embedding = binary_final_logits.to('cpu'), instance_embedding.to('cpu') 
    binary_img = torch.argmax(binary_final_logits, dim=1).squeeze().numpy()
    binary_img[0:65,:] = 0

    # lane clustering & segemented frame embedding
    rbg_emb, cluster_result = process_instance_embedding(instance_embedding, binary_img,
                                                          distance=1.5, lane_num=4)

    rbg_emb = cv2.resize(rbg_emb, dsize=(org_shape[1], org_shape[0]), interpolation=cv2.INTER_LINEAR)
    a = 0.6
    frame = a * gt_img_org[..., ::-1] / 255 + rbg_emb * (1 - a)
    frame = np.rint(frame * 255)
    frame = frame.astype(np.uint8)

    return frame


image = cv2.imread('TUSIMPLE/test_clips/1494452927854312215/1.jpg')
cv2_imshow(inference(image))

In [ ]:
import os.path as ops
import numpy as np
import torch
import cv2
import time
import os
import matplotlib.pylab as plt
import sys
from tqdm import tqdm
import imageio
from dataset.dataset_utils import TUSIMPLE
from Lanenet.model2 import Lanenet
from utils.evaluation import gray_to_rgb_emb, process_instance_embedding
from google.colab.patches import cv2_imshow


def video2segemented_video(video_path):
    # TODO: video to frames
   cap = cv2.VideoCapture(video_path)
   count=0
  
   
   while (count<12):
    ret, frame = cap.read()
    try:
      frame =cv2.resize(frame, (960, 540))
    except cv2.error:
      continue

    if(int(cap.get(1))% 30 ==0):
      print('Saved frame number : '+str(int(cap.get(1))))
      cv2.imwrite("/content/drive/MyDrive/frames/frame%d.jpg" % count, frame) 
      if cv2.waitKey(42)==ord('q'):
        break   
      count+=1  
      print("count+1") 
   cap.release()
   cv2.destroyAllWindows()
    # TODO: extract lane from frame

   count=0

   while(count<12):
     image = cv2.imread('/content/drive/MyDrive/frames/frame%d.jpg' % count)
     image=inference(image)
     try:
      cv2.imwrite("/content/drive/MyDrive/nframes/frame%d.jpg" % count, image)
     except cv2.error:
      continue
     print("%d downloaded" % count)
     count+=1


    # TODO: frames to video & store video
   import re
   path = "/content/drive/MyDrive/nframes"
   paths = [os.path.join(path , i ) for i in os.listdir(path) if re.search(".jpg$", i )]

   store1 = []
   store2 = []
   for i in paths :
      if len(i) == 19 :
          store2.append(i)
      else :
          store1.append(i)

   paths = list(np.sort(store1)) + list(np.sort(store2))

   pathIn= '/content/drive/MyDrive/'
   pathOut = '/content/drive/MyDrive/out.mp4'
   fps = 1
   frame_array = []
   for idx , path in enumerate(paths) : 
    if (idx % 2 == 0) | (idx % 5 == 0) :
        continue
    img = cv2.imread(path)
    height, width, layers = img.shape
    size = (width,height)
    frame_array.append(img)
   out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
   for i in range(len(frame_array)):    
    out.write(frame_array[i])
   out.release()

    

In [ ]:
# Test
video_path = "/content/drive/MyDrive/video_example_Trim.mp4" # input your video path
video2segemented_video(video_path)